In [ ]:
pip install langdetect

In [ ]:
from os import path
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import pylab as py
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
from tqdm import tqdm
from scipy import sparse
import csv, random, numpy, os, re, nltk, scipy, gensim
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from langdetect import detect
from sklearn.ensemble import RandomForestClassifier
from csv import DictReader
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
import math
import re
from collections import Counter
# import score

In [ ]:
class DataSet():
    def __init__(self, path="../input/d/abdullahalmazid/data-mazid/"):
        self.path = path

        print("Reading dataset")
        bodies = "train_bodies_zv1.csv"
        stances = "train_stances_zv2.csv"

        self.stances = self.read(stances)
        self.stances_v2 = self.read(stances)
        articles = self.read(bodies)
        self.articles = dict()

        #make the body ID an integer value
        for s in self.stances:
            s['Body ID'] = int(s['Body ID'])

        #copy all bodies into a dictionary
        for article in articles:
            self.articles[int(article['Body ID'])] = article['articleBody']
            
        #make the body ID an integer value
        for s in self.stances_v2:
            s['Body'] = self.articles[int(s['Body ID'])]
            

        print("Total stances: " + str(len(self.stances)))
        print("Total bodies: " + str(len(self.articles)))



    def read(self,filename):
        rows = []
        with open(self.path + "/" + filename, "r", encoding='utf-8') as table:
            r = DictReader(table)

            for line in r:
                rows.append(line)
        return rows

In [ ]:
dataset = DataSet()

In [ ]:
WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [ ]:
def read_ids(file,base="../input/idsmazid"):
    ids = []
    with open(base+"/"+file,"r") as f:
        for line in f:
            ids.append(int(line))
        return ids
    
def split(dataset, base="../input/idsmazid"):
    if not (os.path.exists(base+"/"+"training_ids_zv1.txt")
            and os.path.exists(base+"/"+"dev_ids_zv1.txt") and os.path.exists(base+"/"+"test_ids_zv1.txt")):
        raise Exception("There is an error and the dataset reader cannot find the "
                        "{training_ids|test_ids|dev_ids}.txt file. Please make sure your python paths "
                        "are configured correctly")

    training_ids = read_ids("training_ids_zv1.txt",base)
    dev_ids = read_ids("dev_ids_zv1.txt",base)
    test_ids = read_ids("test_ids_zv1.txt",base)
    
    print (test_ids)
    #return the stances that meet these criteria
    training_stances = []
    dev_stances = []
    test_stances = []
    test_datapoint_index = []
    
    index=0;
    for stance in dataset.stances:
        if stance['Body ID'] in training_ids:
            training_stances.append(stance)
        if stance['Body ID'] in dev_ids:
            dev_stances.append(stance)
        if stance['Body ID'] in test_ids:
            test_stances.append(stance)
            test_datapoint_index.append(index)
        
        index = index + 1

    return {"training":training_stances, "dev":dev_stances, "test": test_stances, "test_datapoint_index": test_datapoint_index}

In [ ]:
print("\n[1] Loading data..")
data_splits = split(dataset)
# in the format: Stance, Headline, BodyID
training_data = data_splits['training']
dev_data = data_splits['dev']
test_data = data_splits['test'] # currently 0 test points
test_datapoint_index = data_splits['test_datapoint_index']
# Change the number of training examples used.
N = int(len(training_data) * 1.0)
training_data = training_data[:N]

print("\t-Training size:\t", len(training_data))
print("\t-Dev size:\t", len(dev_data))
print("\t-Test data:\t", len(test_data)) #184
#for test data id starts at 31 and ends at 45
len(dataset.stances)
len(test_datapoint_index)

In [ ]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def calculateUnrelatedStanceAccuracy_v1(threshold, rangeN):
    matchCnt = 0
    unrelatedCnt=0
    ypred=np.zeros(rangeN)
    minm = 0
    unmatch = 0
    relatedCnt = 0
    mat = np.zeros((2,2))

    for i in range(0,rangeN):
        vector1 = text_to_vector(dataset.stances_v2[i]['Headline'])
        vector2 = text_to_vector(dataset.stances_v2[i]['Body'])

        cos_scores = get_cosine(vector1, vector2)
        #cos_scores = jaccard_similarity(vector1, vector2)
        #print(dataset.stances_v2[i]['Stance'])
        if dataset.stances_v2[i]['Stance'] == 'unrelated':
            #print(cos_scores)
            unrelatedCnt = unrelatedCnt + 1
        else:
            #print(cos_scores)
            relatedCnt += 1
            if cos_scores < threshold:
                minm += 1
            
        if cos_scores < threshold:
            ypred[i]= 1
            if dataset.stances_v2[i]['Stance'] == 'unrelated':
               mat[0][0] += 1
               matchCnt+= 1
            else:
               mat[0][1] += 1
        else:
            if dataset.stances_v2[i]['Stance'] != 'unrelated':
               mat[1][1] += 1
               unmatch += 1
            else:
               mat[1][0] += 1
            
        
    #print(unrelatedCnt)  
    mat[0][0] = mat[0][0]/unrelatedCnt
    mat[0][1] = mat[0][1]/relatedCnt
    mat[1][0] = mat[1][0]/unrelatedCnt
    mat[1][1] = mat[1][1]/relatedCnt
    print(minm)
    return matchCnt/unrelatedCnt,unmatch/relatedCnt,mat,ypred

In [ ]:
 # jaccard_similarity optimal point- (threshold: 0.0755, accuracy: 0.8830065359477124)
 # get_cosine - (threshold: 0.227, accuracy: 0.8911764705882353)

gAccuracyForUnrelatedStances,gAccuracyForRelatedStances,mat,unrelatedPred = calculateUnrelatedStanceAccuracy_v1(0.22, len(dataset.stances_v2))
print ("Accuracy: ",gAccuracyForUnrelatedStances)
print ("Accuracy: ",gAccuracyForRelatedStances)
print ( )
print(unrelatedPred[3])
print ( )
labels = ["Unrelated","Related"]
fig, ax = plt.subplots(figsize=(11,11))  
sns.heatmap(mat, square=True, annot=True, cbar=True,cmap="icefire",fmt=".2%",
            xticklabels=labels,yticklabels=labels,linewidths=.3, ax=ax)
    
plt.xlabel('True label')
plt.ylabel('Predicted label');

In [ ]:
# Get the bodies of training data points
def get_bodies(data):
    bodies = []
    for i in range(len(data)):
        bodies.append(dataset.articles[data[i]['Body ID']])
    return bodies

# Get the headlines of training data points
def get_headlines(data):
    headlines = []
    for i in range(len(data)):
        headlines.append(data[i]['Headline'])
    return headlines

In [ ]:
# Get bodies and headlines for dev and training data
training_bodies = get_bodies(training_data)
training_headlines = get_headlines(training_data)
dev_bodies = get_bodies(dev_data)
dev_headlines = get_headlines(dev_data)
test_bodies = get_bodies(test_data)
test_headlines = get_headlines(test_data)

print(len(training_bodies))
print(len(training_headlines))
print(len(dev_bodies))
print(len(dev_headlines))
print(len(test_bodies))
print(len(test_headlines))

In [ ]:
# Function for extracting tf-idf vectors (for both the bodies and the headlines).
# https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments

def extract_tfidf(training_headlines, training_bodies, dev_headlines="", dev_bodies="", test_headlines="", test_bodies=""):
    # Body vectorisation
    body_vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, stop_words='english')#, max_features=1024)
    bodies_tfidf = body_vectorizer.fit_transform(training_bodies)

    # Headline vectorisation
    headline_vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True, stop_words='english')#, max_features=1024)
    headlines_tfidf = headline_vectorizer.fit_transform(training_headlines)

    # Tranform dev/test bodies and headlines using the trained vectorizer (trained on training data)
    bodies_tfidf_dev = body_vectorizer.transform(dev_bodies)
    headlines_tfidf_dev = headline_vectorizer.transform(dev_headlines)

    bodies_tfidf_test = body_vectorizer.transform(test_bodies)
    headlines_tfidf_test = headline_vectorizer.transform(test_headlines)
    
    feature_names = np.array(body_vectorizer.get_feature_names())
    sorted_by_idf = np.argsort(body_vectorizer.idf_) 
    print('Features with lowest and highest idf in the body vector:\n')
    # The token which appears maximum times but it is also in all documents, has its idf the lowest
    print("Features with lowest idf:\n{}".format(
    feature_names[sorted_by_idf[:10]]))
    # The tokens can have the most idf weight because they are the only tokens that appear in one document only
    print("\nFeatures with highest idf:\n{}".format(
    feature_names[sorted_by_idf[-10:]]))

    # Combine body_tfdif with headline_tfidf for every data point. 
    training_tfidf = scipy.sparse.hstack([bodies_tfidf, headlines_tfidf])
    dev_tfidf = scipy.sparse.hstack([bodies_tfidf_dev, headlines_tfidf_dev])
    test_tfidf = scipy.sparse.hstack([bodies_tfidf_test, headlines_tfidf_test])

    return training_tfidf, dev_tfidf, test_tfidf

In [ ]:
lemmatizer = nltk.WordNetLemmatizer()

# Tokenisation, Normalisation, Capitalisation, Non-alphanumeric removal, Stemming-Lemmatization
def preprocess(string):
    # to lowercase, non-alphanumeric removal
    step1 = " ".join(re.findall(r'\w+', string, flags=re.UNICODE)).lower()
    step2 = [lemmatizer.lemmatize(t).lower() for t in nltk.word_tokenize(step1)]

    return step2

# Function for extracting word overlap
def extract_word_overlap(headlines, bodies):
    word_overlap = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        preprocess_headline = preprocess(headline)
        preprocess_body = preprocess(body)
        
        # Lenght of common words b/w body and headline / Length of all the words of body & headline
        features = len(set(preprocess_headline).intersection(preprocess_body)) / float(len(set(preprocess_headline).union(preprocess_body)))
        word_overlap.append(features)
        
        # Convert the list to a sparse matrix (in order to concatenate the cos sim with other features)
        word_overlap_sparse = scipy.sparse.coo_matrix(numpy.array(word_overlap)) 
    return word_overlap_sparse

In [ ]:
# Function for extracting the cosine similarity between bodies and headlines. 
def extract_cosine_similarity(headlines, bodies):
    vectorizer = TfidfVectorizer(ngram_range=(1,2), lowercase=True, stop_words='english')#, max_features=1024)
    
    cos_sim_features = []
    for i in range(0, len(bodies)):
        body_vs_headline = []
        body_vs_headline.append(bodies[i])
        body_vs_headline.append(headlines[i])
        tfidf = vectorizer.fit_transform(body_vs_headline)
        
        cosine_similarity = (tfidf * tfidf.T).A
        cos_sim_features.append(cosine_similarity[0][1])

    # Convert the list to a sparse matrix (in order to concatenate the cos sim with other features)
    cos_sim_array = scipy.sparse.coo_matrix(numpy.array(cos_sim_features)) 

    return cos_sim_array

In [ ]:
# Function for combining features of various types (lists, coo_matrix, np.array etc.)
def combine_features(tfidf_vectors, cosine_similarity, word_overlap):
    combined_features =  sparse.bmat([[tfidf_vectors, word_overlap.T, cosine_similarity.T]])
    return combined_features

In [ ]:
# Function for extracting features
# Feautres: 1) Word Overlap, 2) TF-IDF vectors, 3) Cosine similarity, 4) Word embeddings
def extract_features(train, dev, test):
# Get bodies and headlines for dev and training data
    training_bodies = get_bodies(training_data)
    training_headlines = get_headlines(training_data)
    dev_bodies = get_bodies(dev_data)
    dev_headlines = get_headlines(dev_data)
    test_bodies = get_bodies(test_data)
    test_headlines = get_headlines(test_data)

    # Extract tfidf vectors
    print("\t-Extracting tfidf vectors..")
    training_tfidf, dev_tfidf, test_tfidf = extract_tfidf(training_headlines, training_bodies, dev_headlines, dev_bodies, test_headlines, test_bodies)
    print("\t-Tfidf vectors extracted..")

    # Extract word overlap 
    print("\t-Extracting word overlap..")
    training_overlap = extract_word_overlap(training_headlines, training_bodies)
    dev_overlap = extract_word_overlap(dev_headlines, dev_bodies)
    test_overlap = extract_word_overlap(test_headlines, test_bodies)
    print("\t-Word overlap extracted..")

#     # Extract cosine similarity between bodies and headlines. 
    print("\t-Extracting cosine similarity..")
    training_cos = extract_cosine_similarity(training_headlines, training_bodies)
    dev_cos = extract_cosine_similarity(dev_headlines, dev_bodies)
    test_cos = extract_cosine_similarity(test_headlines, test_bodies)
    print("\t-Cosine similarity extracted..")

    # Combine the features
    training_features = combine_features(training_tfidf, training_cos, training_overlap)
    dev_features = combine_features(dev_tfidf, dev_cos, dev_overlap)
    test_features = combine_features(test_tfidf, test_cos, test_overlap)
    print("\t-Combined features returned..")

    return training_features, dev_features, test_features

In [ ]:
# Feature extraction
print("[2] Extracting features.. ")
# extract_features(training_data, dev_data, test_data)
training_features, dev_features, test_features = extract_features(training_data, dev_data, test_data)

In [ ]:
classes_v2 = ['agree', 'disagree', 'discuss', 'unrelated']

def report_score_v2(test,pred,algo):
    #accuracy calculation
    accuracy = accuracy_score(test,pred)
    
    #print('\n Accuracy_score for %s = %s \n'%(algo,accuracy))
    
    #confusion_matrix
    mat = confusion_matrix(test,pred)
    
    #print ( 'Confusion Matrix Start' )
    #print ( mat )
    #print ( 'Confusion Matrix End' )
    
    agreeAC = mat[0][0]/(mat[0][0]+mat[0][1]+mat[0][2]+mat[0][3])
    disagreeAC = mat[1][1]/(mat[1][0]+mat[1][1]+mat[1][2]+mat[1][3])
    discussAC = mat[2][2]/(mat[2][0]+mat[2][1]+mat[2][2]+mat[2][3])
    unrelatedAC = gAccuracyForUnrelatedStances

    print ( "Total Accuracy: ",100*(agreeAC + disagreeAC + unrelatedAC) /3 )
    
    fig, ax = plt.subplots(figsize=(11,11))  
    ax.set_title("Confusion Matrix for %s" %algo)
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=True,cmap="icefire",
                xticklabels=classes,yticklabels=classes,linewidths=.3, ax=ax,annot_kws={"fontsize":12})
    plt.xlabel('True label')
    plt.ylabel('Predicted label');

    #classification report
    cls = classification_report(test,pred, target_names=classes)
    print(cls)

In [ ]:
classes = ['agree', 'disagree', 'discuss']
classes2 = ['agree', 'disagree', 'discuss','unrelated']

def report_score(test,pred,algo):
    #accuracy calculation
    accuracy = accuracy_score(test,pred)
    print('\n Accuracy_score for %s = %s \n'%(algo,accuracy))
    
    #confusion_matrix
    mat = confusion_matrix(test,pred)
    
    print ('Confusion Matrix Start')
    print (mat)
    print ('Confusion Matrix End')
    
    agreeAC = mat[0][0]/(mat[0][0]+mat[0][1]+mat[0][2]+mat[0][3])
    disagreeAC = mat[1][1]/(mat[1][0]+mat[1][1]+mat[1][2]+mat[1][3])
    discussAC = mat[2][2]/(mat[2][0]+mat[2][1]+mat[2][2]+mat[2][3])
    unrelatedAC = gAccuracyForUnrelatedStances
    
    rel = np.zeros((3,3))
    
    for i in range(0,3):
        for j in range(0,3):
            rel[i][j] = mat[i][j]/(mat[i][0]+mat[i][1]+mat[i][2]+mat[i][3])
    
    

    
    print ('New AC start')
    print("Agree Accuracy: ",agreeAC*100)
    print("Disagree Accuracy: ",disagreeAC*100)
    print("Discuss Accuracy: ",discussAC*100)
    print("unrelated Accuracy: ",unrelatedAC*100)
    print("-------------------------------------")
    print ("Total Accuracy: ",100*(agreeAC + disagreeAC + unrelatedAC) /3)
    print ('New AC End')
    print(rel)
    fig, ax = plt.subplots(figsize=(11,11))  
    ax.set_title("Confusion Matrix for %s" %algo)
    sns.heatmap(rel, square=True, annot=True, cbar=True,cmap="icefire",fmt=".2%",
                xticklabels=classes,yticklabels=classes,linewidths=.3, ax=ax,annot_kws={"fontsize":20})
    
    plt.xlabel('True label')
    plt.ylabel('Predicted label');

    #classification report
    cls = classification_report(test,pred, target_names=classes2)
    print(cls)

In [ ]:
def process_output(y_pred_lr,unrelatedPred):
    for i in range(0,len(y_pred_lr)):
        if y_pred_lr[i] != "unrelated" and unrelatedPred[test_datapoint_index[i]] == 1:
            y_pred_lr[i] = "unrelated"
    return y_pred_lr

In [ ]:
# Creating targets
targets_tr = [a['Stance'] for a in training_data]
targets_dev = [a['Stance'] for a in dev_data]
targets_test = [a['Stance'] for a in test_data]

In [ ]:
def change(f):
    if f == 'unrelated':
        return 0
    elif f == 'disagree':
        return 1
    elif f == 'discuss':
        return 2
    elif f == 'agree':
        return 3
    else:
        return 0

y = [change(x) for x in targets_tr]
y_test = [change(x) for x in targets_test]

In [ ]:
def predict(self, text):
        predicted = self.logR_pipeline.predict([text])
        predicedProb = self.logR_pipeline.predict_proba([text])[:,1]
        return bool(predicted), float(predicedProb)
    
#print (dataset.stances_v2[32])

In [ ]:
# Fitting model
print("[3] Fitting model..")
print("\t-Logistic Regression")
lr = LogisticRegression(penalty='elasticnet',C = 0.4, class_weight='balanced', solver='saga', max_iter=150, l1_ratio=0.7) 
y_pred_lr = lr.fit(training_features, targets_tr).predict(test_features)
#print (unrelatedPred)
#print(len(y_pred_lr))
#print(len(unrelatedPred))
#y_pred_lr = process_output(y_pred_lr,unrelatedPred)

# Evaluation
print("[4] Evaluating model..")
report_score(targets_test, y_pred_lr,'Logistic Regression')
print("\t-Done with Logistic Regression")

#highest:55.17%

In [ ]:
print("\t-Random Forest Classifier")
rf = RandomForestClassifier(n_estimators=20, random_state=19,criterion='gini',min_samples_leaf=1)
y_pred_rf = rf.fit(training_features, targets_tr).predict(test_features)
#y_pred_rf= process_output(y_pred_rf,unrelatedPred)
print("[4] Evaluating model..")
report_score(targets_test, y_pred_rf,'Random Forest Classifier')
print("\t-Done with Random Forest Classifier")

In [ ]:
print("[3] Fitting model..")
print("\t-Multinomial Naive Bayes")
nb = MultinomialNB(alpha=0.09, class_prior=(0.5,0.5,0.1,0.0), fit_prior=True)
# nb = MultinomialNB(alpha=0.09, class_prior=(0.5,0.5,0.1,0.0), fit_prior=True) best
y_pred_nb = nb.fit(training_features, targets_tr).predict(test_features)
#y_pred_nb= process_output(y_pred_nb,unrelatedPred)

print("[4] Evaluating model..")
report_score(targets_test, y_pred_nb,'Multinomial Naive Bayes')
print("\t-Done with Multinomial Naive Bayes")

In [ ]:
print("[3] Fitting model..")
print("\t-XGBoost Classifier")
xgb = XGBClassifier(learning_rate=0.05,n_estimators=100,random_state=2,subsample=1.0,warm_start=True)
y_pred_xgb = xgb.fit(training_features, targets_tr).predict(test_features)
#y_pred_xgb = process_output(y_pred_xgb, unrelatedPred)
# Evaluation
print("[4] Evaluating model..")
report_score(targets_test, y_pred_xgb,'XGBoost')
print("\t-Done with XGBoost Classifier")

In [ ]:
y_pred_ensemble2 = []

stances = {"agree":0,"disagree":1,"discuss":2,"unrelated":3}
class_id = {0:"agree",1:"disagree",2:"discuss",3:"unrelated"}
weights = [0.1,0.5,0.3,1.9]

#Assign weights dyanamically based on the degree to which they could predict the stances
weight_lr = {"agree":0.3762,"disagree":0.3077,"discuss":1.15,"unrelated":0.0}
weight_nb = {"agree":0.7738,"disagree":0.7289,"discuss":0.3588,"unrelated":0.0}
weight_rf = {"agree":0.9786,"disagree":0.0526,"discuss":0.0588,"unrelated":0.0}
weight_xgb = {"agree":0.7143,"disagree":0.8276, "discuss":0.0588,"unrelated":0.0}

for i in range(len(targets_test)):
  classp = np.zeros(4)
  maxg = 0
  max_weight = 0
  
  classp[stances[y_pred_lr[i]]] += weight_lr[y_pred_lr[i]]
  classp[stances[y_pred_rf[i]]] += weight_rf[y_pred_rf[i]]
  classp[stances[y_pred_nb[i]]] += weight_nb[y_pred_nb[i]]
  classp[stances[y_pred_xgb[i]]] += weight_xgb[y_pred_xgb[i]]
    
  for j in range(len(classp)):
    if classp[j] > max_weight:
      maxg = j
      max_weight = classp[j]
  y_pred_ensemble2.append(class_id[maxg])

#y_pred_ensemble2 = process_output(y_pred_ensemble2,unrelatedPred)
#print("Accuracy:",accuracy_score(targets_test,y_pred_ensemble2)*100)
report_score(targets_test, y_pred_ensemble2,'Ensemble')

In [ ]:
#weighted accuracy computation
weight_en = {"agree": 5,"disagree": 1,"discuss": 0.2,"unrelated": 0.2}
accuracy_en = 0.0
accurate_weight_total = 0.0
weight_total = 0.0

for i in range(len(y_pred_ensemble2)):
    if targets_test[i] != "unrelated":
        weight_total += weight_en[targets_test[i]]
        if targets_test[i] == y_pred_ensemble2[i]:
            accurate_weight_total += weight_en[targets_test[i]]
             
print (accurate_weight_total/weight_total)